In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
from pandas.core.indexes.api import InvalidIndexError
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import cv2
import os
import requests
import torch
import argparse
import torchvision
from numpy import expand_dims
from tensorflow.keras.utils import load_img
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from torchvision import transforms
#import pickle
#import torch.nn as nn
from torchvision.models import detection
import torchvision.models as models
import torchvision.transforms as transforms
#from torch.autograd import Variable
#from sklearn.metrics import jaccard_similarity_score
from scipy import spatial
#from sklearn.decomposition import PCA as RandomizedPCA
from PIL import Image
import glob
#pd.set_option('display.max_columns', None)
from google.colab.patches import cv2_imshow

In [4]:
root_path = 'gdrive/My Drive/coco_minitrain_25k/' 

In [ ]:
#Displaying a single image from the folder that contains the images of Class Person
image = plt.imread('/content/gdrive/MyDrive/coco_minitrain_25k /images/images_person/000000104747.jpg')
plt.imshow(image)


In [ ]:
!pip install tensorflow

**OBJECT DETECTION USING THE YOLOV3 MODEL**

**Performing object detection using two models namely YOLOv3 and Faster RCNN. The feature extraction and the similarity search is done on the basis of the Faster RCNN model.**

In [ ]:
# Performing Object Detection bu using YOLOV3 model

import numpy as np
from numpy import expand_dims
from tensorflow.keras.utils import load_img
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle
 
class BoundBox:
	def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
		self.xmin = xmin
		self.ymin = ymin
		self.xmax = xmax
		self.ymax = ymax
		self.objness = objness
		self.classes = classes
		self.label = -1
		self.score = -1
 
	def get_label(self):
		if self.label == -1:
			self.label = np.argmax(self.classes)
 
		return self.label
 
	def get_score(self):
		if self.score == -1:
			self.score = self.classes[self.get_label()]
 
		return self.score
 
def _sigmoid(x):
	return 1. / (1. + np.exp(-x))
 
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
	grid_h, grid_w = netout.shape[:2]
	nb_box = 3
	netout = netout.reshape((grid_h, grid_w, nb_box, -1))
	nb_class = netout.shape[-1] - 5
	boxes = []
	netout[..., :2]  = _sigmoid(netout[..., :2])
	netout[..., 4:]  = _sigmoid(netout[..., 4:])
	netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
	netout[..., 5:] *= netout[..., 5:] > obj_thresh
 
	for i in range(grid_h*grid_w):
		row = i / grid_w
		col = i % grid_w
		for b in range(nb_box):
			objectness = netout[int(row)][int(col)][b][4]
			if(objectness.all() <= obj_thresh): continue
			# The first 4 elements are x, y, w, and h
			x, y, w, h = netout[int(row)][int(col)][b][:4]
			x = (col + x) / grid_w 
			y = (row + y) / grid_h 
			w = anchors[2 * b + 0] * np.exp(w) / net_w 
			h = anchors[2 * b + 1] * np.exp(h) / net_h 
			# Class Probabilities
			classes = netout[int(row)][col][b][5:]
			box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
			boxes.append(box)
	return boxes
 
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
	new_w, new_h = net_w, net_h
	for i in range(len(boxes)):
		x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
		y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
		boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
		boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
		boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
		boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)
 

def _interval_overlap(interval_a, interval_b):
	x1, x2 = interval_a
	x3, x4 = interval_b
	if x3 < x1:
		if x4 < x1:
			return 0
		else:
			return min(x2,x4) - x1
	else:
		if x2 < x3:
			 return 0
		else:
			return min(x2,x4) - x3


def bbox_iou(box1, box2):
	intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
	intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
	intersect = intersect_w * intersect_h
	w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
	w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
	union = w1*h1 + w2*h2 - intersect
	return float(intersect) / union
 
def do_nms(boxes, nms_thresh):
	if len(boxes) > 0:
		nb_class = len(boxes[0].classes)
	else:
		return
	for c in range(nb_class):
		sorted_indices = np.argsort([-box.classes[c] for box in boxes])
		for i in range(len(sorted_indices)):
			index_i = sorted_indices[i]
			if boxes[index_i].classes[c] == 0: continue
			for j in range(i+1, len(sorted_indices)):
				index_j = sorted_indices[j]
				if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
					boxes[index_j].classes[c] = 0
 
# Loading an image
filename="/content/gdrive/MyDrive/coco_minitrain_25k /images/images_person/000000104747.jpg"

def load_image_pixels(filename, shape):
	image = load_img(filename)
  #newsize=(800,800)
  #image=image.resize(newsize)
	width, height = image.size
	image = load_img(filename, target_size=shape)
	image = img_to_array(image)
	image = image.astype('float32')
	image /= 255.0
	image = expand_dims(image, 0)
	return image, width, height
 
# The results above a threshold
def get_boxes(boxes, labels, thresh):
	v_boxes, v_labels, v_scores = list(), list(), list()
	for box in boxes:
		for i in range(len(labels)):
			# Checking if the threshold value for the label is high enough
			if box.classes[i] > thresh:
				v_boxes.append(box)
				v_labels.append(labels[i])
				v_scores.append(box.classes[i]*100)
	return v_boxes, v_labels, v_scores
 
# Drawing the bounding boxes around the objects detected
def draw_boxes(filename, v_boxes, v_labels, v_scores):
	data = pyplot.imread(filename)
	pyplot.imshow(data)
	ax = pyplot.gca()
	# Plotting each bounding box
	for i in range(len(v_boxes)):
		box = v_boxes[i]
		y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
		width, height = x2 - x1, y2 - y1
		rect = Rectangle((x1, y1), width, height, fill=False, color='blue')
		ax.add_patch(rect)
		label = "%s (%.3f)" % (v_labels[i], v_scores[i])
		pyplot.text(x1, y1, label, color='yellow')
	pyplot.show()
 
# Loading the yolov3 model
model = load_model('/content/gdrive/MyDrive/coco_minitrain_25k /images/yolo.h5')
input_w, input_h = 416,416
photo_filename = '/content/gdrive/MyDrive/coco_minitrain_25k /images/images_person/000000104747.jpg'
image, image_w, image_h = load_image_pixels(photo_filename, (input_w, input_h))
# Making prediction
yhat = model.predict(image)
print([a.shape for a in yhat])
# Defining the anchors
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]

# Defining the probability threshold for the detected objects
class_threshold = 0.6
boxes = list()
for i in range(len(yhat)):
	boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)
correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)

# Suppressing the  non-maximal boxes
do_nms(boxes, 0.5)

# Defining the labels to be detected in an image
labels = ["person"]

v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)

for i in range(len(v_boxes)):
	print(v_labels[i], v_scores[i])

#Plotting the bounding box, mentioning the label and the score of the detected object
draw_boxes(photo_filename, v_boxes, v_labels, v_scores)

**LOADING A PRETRAINED FASTER RCNN MODEL TO PERFORM OBJECT DETECTION AND EXTRACTING THE BOUNDING BOXES**

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, pretrained_backbone=True)
model.eval()

In [ ]:
url = 'https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/'
html = requests.get(url).content
df_data= pd.read_html(html)

cls = df_data[0][['ID','Object (2017 Rel.)']]
colors = np.random.uniform(0, 255, size=(len(cls), 3))

In [ ]:
img = cv2.imread("/content/gdrive/My Drive/coco_minitrain_25k /images/images_person/000000104747.jpg")
img = cv2.resize(img,(400,400))
copy_img = img.copy()
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, axis=0)
img = img / 255.0
img= torch.FloatTensor(img)
img_detect = model(img)[0]

img_detect

**DISPLAYING AN IMAGE WITH BOUNDING BOX USING A FASTER RCNN PRETRAINED MODEL**

In [ ]:
for j in range(0, len(img_detect["boxes"])):
	confid = img_detect["scores"][j]
	if confid >.90 and int(img_detect["labels"][j])==1:
		idx = int(img_detect["labels"][j])
		box = img_detect["boxes"][j].detach().cpu().numpy()
		(sX, sY, lX, lY) = box.astype("int")

		label = "{}: {:.2f}%".format(list(cls[cls.ID==idx]['Object (2017 Rel.)'].values)[0], confid * 100)
		print(label)
		cv2.rectangle(copy_img, (sX, sY), (lX, lY),colors[idx], 2)
		cv2_imshow(cv2.resize(copy_img[sY:lY,sX:lX],(50,50)))
cv2_imshow(copy_img)

**CALCULATING IOU VALUE FOR AN IMAGE**

In [ ]:
img = cv2.imread("/content/gdrive/MyDrive/coco_minitrain_25k /images/images_person/000000021895.jpg")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Creating a bounding box for an image
first_bb_points = [[250, 210], [440, 210], [440, 390], [250, 390]]
stencil = np.zeros(img.shape).astype(img.dtype)
contours = [np.array(first_bb_points)]
color = [255, 255, 255]
cv2.fillPoly(stencil, contours, color)
result1 = cv2.bitwise_and(img, stencil)
result1 = cv2.cvtColor(result1, cv2.COLOR_BGR2RGB)
plt.imshow(result1)

# Second bounding box
second_bb_points = [[280, 190], [438, 190], [438, 390], [280, 390]]
stencil = np.zeros(img.shape).astype(img.dtype)
contours = [np.array(second_bb_points)]
color = [255, 255, 255]
cv2.fillPoly(stencil, contours, color)
result2 = cv2.bitwise_and(img, stencil)
result2 = cv2.cvtColor(result2, cv2.COLOR_BGR2RGB)
plt.imshow(result2)

# IoU calculation
intersection = np.logical_and(result1, result2)
union = np.logical_or(result1, result2)
iou_score = np.sum(intersection) / np.sum(union)
print('IoU is: ', iou_score)

In [ ]:
img_path = '/content/gdrive/MyDrive/coco_minitrain_25k /images/images_person'
list_person = glob.glob(os.path.join(img_path, "*.jpg"))
df_person = pd.DataFrame(columns = ['boxes','labels','scores','Image_name','object_feature_tensor'])


In [ ]:
p=0
for i in list_person:
  img = cv2.imread(i)
  img = cv2.resize(img,(300,300))
  copy_img = img.copy()
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = img.transpose((2, 0, 1))
  img = np.expand_dims(img, axis=0)
  img = img / 255.0
  img = torch.FloatTensor(img)
  img_detect = model(img)[0]
  for keys in img_detect:
    img_detect[keys] = img_detect[keys].tolist()
  t =  pd.DataFrame.from_dict(img_detect)
  t['Image_name'] = i.split("/")[-1]
  t = t[(t.labels==1) & (t.scores>0.90)]

  df_person = df_person.append(t)
  #p = p+1
  #if p%100==0:
    #print(p)
  

In [ ]:
#df_person.head()
#final_df.iloc[0]['Image_name']

In [ ]:
p=0
ext_features = []

for index,row in df_person.iterrows():
  img = cv2.imread(img_path+"/"+row['Image_name'])

  box = torch.Tensor(row["boxes"]).detach().cpu().numpy()
  (sX, sY, lX, lY) = box.astype("int")
  person_detect = img[sY:lY,sX:lX]
  person_detect = cv2.resize(person_detect,(100,100))

  person_detect = cv2.cvtColor(person_detect, cv2.COLOR_BGR2RGB)
  person_detect= person_detect.transpose((2, 0, 1))
  person_detect = np.expand_dims(person_detect, axis=0)
  person_detect = person_detect / 255.0
  person_detect = torch.FloatTensor(person_detect)
  
  print(person_detect)

  image, _= model.transform(person_detect, None)
  features = model.backbone(image.tensors)['pool']
  ext_features.append(features.detach().cpu().numpy()[0])
  p = p+1
  if p%100==0:
    print(p)
df_person['object_feature_tensor'] = ext_features

In [ ]:
df_person.head(20)

df_person['ID'] = df_person.index.to_list()

**EXECUTING SIMILARITY SEARCH** 

In [ ]:
def calculate_similarity(v1, v2):
 return 1 - spatial.distance.cosine(v1, v2)

query_list = [500,1264,5264,8000,11528,16000,19160]
v=dict()
for i in query_list:
  t_f = []
  t_df = df_person.copy()
  feat_tesn = torch.FloatTensor(df_person['object_feature_tensor'].iloc[i].flatten())
  for index,row in df_person.iterrows():
    if len(str(row['object_feature_tensor']))!=1:
      t_f.append(calculate_similarity(feat_tesn,torch.FloatTensor(row['object_feature_tensor'].flatten())))
    else:
      t_f.append(0)

  df_person['similarity'] = t_f
  t_df = df_person
  t_df.sort_values(by = 'similarity',ascending = False,inplace = True)
  print(i,t_df.head(10)['ID'].unique())
  v[i] = t_df.head(10)['ID'].unique()

  """
  for index,row in df_person.iterrows():
    t_f.append(calculate_similarity(feat_tesn,torch.FloatTensor(row['object_feature_tensor'].flatten())))
  df_person['similarity'] = t_f
  #temp_df.sort_values(by = 'similarity', )

df_person.sort_values(by = 'similarity').tail(10)
"""

In [ ]:
def image_person(s):
   im_path = img_path+"/"+df_person.iloc[11]['Image_name']
   img = cv2.imread(im_path)
   img_copy = img.copy()
   img = cv2.resize(img,(500,500))
   cv2_imshow(img)
   box = torch.Tensor(df_person.iloc[11]["boxes"]).detach().cpu().numpy()
   (sX, sY, lX, lY) = box.astype("int")
   person_detect = image[sY:lY,sX:lX]
   person_detect = cv2.resize(person_detect,(100,100))
   cv2_imshow(person_detect)

In [ ]:
q=1
for i in query_list:
  print('Query Image No:',q)
  image_person(i)
  q=q+1
  for j in list(v[i]):
    image_person(j)
     

In [ ]:
calculate_similarity(torch.FloatTensor(df_person['object_feature_tensor'].iloc[0].flatten()),torch.FloatTensor(df_person['object_feature_tensor'].iloc[20].flatten()))

In [ ]:
torch.nn.CosineSimilarity(dim=0)(torch.FloatTensor(df_person['object_feature_tensor'].iloc[0]),torch.FloatTensor(df_person['object_feature_tensor'].iloc[15]))